# Hand Gesture Video Control System

Through this project, we are able to control the Functionalities of YouTube or other Video Playback Applications like VLC, Window Media Player, etc. like <b> Play, Pause, Forward/ Backward Playback, Full Screen and Control the Volume</b> by using our Hand Gestures.

In [1]:
import cv2
import mediapipe as mp
import keyboard

## Function to Predict the Hand Gesture

In [2]:
def prediction(tf, tb, f1, f2, f3, f4, thumbs):
    
    bf1 = False
    bf2 = False
    bf3 = False
    bf4 = False
    bth = False
    
    if f1 > tf:
        bf1 = True
    
    if f2 > tf:
        bf2 = True
    
    if f3 > tf:
        bf3 = True
        
    if f4 > tf:
        bf4 = True
        
    if thumbs > tb:
        bth = True
        
    if bf1 and not(bf2 or bf3 or bf4 or bth):
        return "right"
    
    elif (bf1 and bf2) and not(bf3 or bf4 or bth):
        return "volume up"
    
    elif (bf1 and bf2 and bf3) and not(bf4 or bth):
        return "volume down"
    
    elif (bf1 and bf2 and bf3 and bf4) and not(bth):
        return "volume mute"
    
    elif bf1 and bf2 and bf3 and bf4 and bth:
        return "f"
    
    elif not(bf1 or bf2 or bf3 or bf4 or bth):
        return "left"
    
    elif (bth and bf4) and not(bf1 or bf2 or bf3):
        return "space"
    
    elif (bth and bf4 and bf1) and not(bf2 or bf3):
        return "break"
    
    return -1

### OpenCV Code to Identify the Live Hand Gesture and Press the Key Associated with that Gesture using Keyword Library

In [3]:
mphands = mp.solutions.hands
hands = mphands.Hands()
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

_, frame = cap.read()

h, w, c = frame.shape
tf = 50
tb = 120

j=0

while True:
    
    flag=0
    _, frame = cap.read()
    frame = cv2.flip(frame,1)
    
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    result = hands.process(framergb)
    hand_landmarks = result.multi_hand_landmarks
    
    if hand_landmarks:
        
        idx = 0
        area = 0
        a = 0
        b = 0
        c = 0
        d = 0
        
        for handLMs in hand_landmarks:
            
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h
            
            for lm in handLMs.landmark:
                
                x, y = int(lm.x * w), int(lm.y * h)
                
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y
                    
            curr_area = (x_max - x_min) * (y_max - y_min)
            
            if curr_area > area:
                area = curr_area
                a = x_min
                b = y_min
                c = x_max
                d = y_max
                
            idx += 1
            
            if idx == len(hand_landmarks):
                
                #cv2.rectangle(frame, (a-15, b-15), (c+15, d+15), (0, 255, 0), 2)
                #mp_drawing.draw_landmarks(frame, handLMs, mphands.HAND_CONNECTIONS)
                
                if j%30 == 0:
                    
                    '''roi=frame[b-15:d+15,a-15:c+15]
                    roi = cv2.resize(roi, (200, 200))'''
                    
                    i = 0
                    points = []
                    
                    for land in handLMs.landmark:
                        lx, ly = int(land.x*w),int(land.y*h)
                        points.append([lx,ly])
                        #cv2.putText(frame,str(i),(lx,ly),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1,cv2.LINE_AA)
                        i += 1
                        
                    finger1 = points[5][1] - points[8][1]
                    finger2 = points[9][1] - points[12][1]
                    finger3 = points[13][1] - points[16][1]
                    finger4 = points[17][1] - points[20][1]
                    thumb = abs(points[0][0] - points[4][0])
                    
                    pred = prediction(tf,tb,finger1,finger2,finger3,finger4,thumb)
                    
                    if pred == -1:
                        continue
                    
                    elif pred == "break":
                        flag += 1
                        break
        
                    elif pred != "break":
                        keyboard.press_and_release(pred)
            
#     cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)&0xFF
    if key == ord('q'):
        break

    if flag:
        break
        
    if j == 60:
        j = 0
        
    j += 1 
        
        
cap.release()
cv2.destroyAllWindows()